In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import attila_utils
import matplotlib
matplotlib.rcParams['figure.dpi'] = 150
from IPython.display import set_matplotlib_formats

In [45]:
import truth_sets_aaf as tsa
import downsize_truthset as dstset
import pandas as pd

In [100]:
expm = pd.read_csv('~/projects/bsm/results/2019-04-04-truth-sets-aaf/exp_model.csv',
                   dtype={'lambda': 'category', 'region': 'category', 'sample': 'category', 'vartype': 'category', 'log10s2g': 'category'})

In [105]:
sel_rows = (expm['lambda'] == '0.2') & (expm['log10s2g'] == '-3') & (expm['region'] == 'chr22') & (expm['sample'] == 'mix1') & (expm['vartype'] == 'snp')
df = expm.loc[sel_rows, :]
df.head()

,AAF,count,lambda,log10s2g,ntot,region,sample,vartype
1444,2,23,0.2,-3,70,chr22,mix1,snp
1445,4,15,0.2,-3,70,chr22,mix1,snp
1446,6,10,0.2,-3,70,chr22,mix1,snp
1447,8,6,0.2,-3,70,chr22,mix1,snp
1448,10,4,0.2,-3,70,chr22,mix1,snp


In [66]:
outvcfpath = '/home/attila/projects/bsm/results/2019-03-18-truth-sets/chr22/snp/truthset/aaf/exp_model/lambda_0.2/log10s2g_-3/mix1/2.vcf.gz'
invcfpath = '/home/attila/projects/bsm/results/2019-03-18-truth-sets/chr22/snp/truthset/aaf/unfiltered/mix1/2.vcf.gz'
ssize = 23
dstset.sample_positions(ssize=ssize, invcfpath=invcfpath, outvcfpath=outvcfpath)

,CHROM,POS
772,22,18851313
947,22,20034187
1994,22,23657326
2498,22,25579548
3315,22,27547533
3329,22,27610825
4066,22,29350370
4398,22,30630256
4417,22,30742125
4767,22,31942770


In [62]:
df.tail()

,AAF,count,lambda,log10s2g,ntot,region,sample,vartype
1483,92,0,0.2,-3,70,chr22,mix1,snp
1484,94,0,0.2,-3,70,chr22,mix1,snp
1485,96,0,0.2,-3,70,chr22,mix1,snp
1486,98,0,0.2,-3,70,chr22,mix1,snp
1487,100,0,0.2,-3,70,chr22,mix1,snp


In [65]:
outvcfpath = '/home/attila/projects/bsm/results/2019-03-18-truth-sets/chr22/snp/truthset/aaf/exp_model/lambda_0.2/log10s2g_-3/mix1/92.vcf.gz'
invcfpath = '/home/attila/projects/bsm/results/2019-03-18-truth-sets/chr22/snp/truthset/aaf/unfiltered/mix1/92.vcf.gz'
ssize = 0
dstset.sample_positions(ssize=ssize, invcfpath=invcfpath, outvcfpath=outvcfpath)

,CHROM,POS
